In [1]:
import pandas as pd
import pickle

In [2]:
dataset=pd.read_csv("insurance_pre.csv")

In [3]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [6]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [7]:
Independent=dataset[['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]
Independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [8]:
Dependent=dataset[['charges']]
Dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(Independent,Dependent,test_size=0.30,random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [11]:
scy=StandardScaler()
y_train=scy.fit_transform(y_train)
y_test=scy.transform(y_test)

In [12]:
import pickle
filenameX="scx.pkl"
filenameY="scy.pkl"
pickle.dump(sc,open(filenameX,'wb'))
pickle.dump(scy,open(filenameY,'wb'))
scx=pickle.load(open("scx.pkl",'rb'))
scy=pickle.load(open("scy.pkl",'rb'))

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
param_grid={'kernel':['rbf','poly','sigmoid','linear'],'C':[1,10,100],'gamma':['auto','scale']}
grid=GridSearchCV(SVR(),param_grid,refit=True,verbose=3,n_jobs=-1)
grid.fit(x_train,y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\Users\nandh\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=SVR(), n_jobs=-1,
             param_grid={'C': [1, 10, 100], 'gamma': ['auto', 'scale'],
                         'kernel': ['rbf', 'poly', 'sigmoid', 'linear']},
             verbose=3)

In [14]:
re=grid.cv_results_
y_pred=grid.predict(x_test)
from sklearn.metrics import r2_score
r_score = r2_score(y_test,y_pred)
r_score

0.8847497699151066

In [15]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.081985,0.014077,0.010683,0.001766,1,auto,rbf,"{'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}",0.853776,0.819933,0.840614,0.835537,0.764302,0.822832,0.031207,2
1,0.102946,0.010026,0.004734,0.002708,1,auto,poly,"{'C': 1, 'gamma': 'auto', 'kernel': 'poly'}",0.847242,0.807637,0.822310,0.818787,0.768078,0.812811,0.025843,5
2,0.086598,0.011042,0.018045,0.005601,1,auto,sigmoid,"{'C': 1, 'gamma': 'auto', 'kernel': 'sigmoid'}",-34.369784,-43.032200,-35.252815,-44.328836,-24.609005,-36.318528,7.089213,20
3,0.094331,0.010183,0.003441,0.001841,1,auto,linear,"{'C': 1, 'gamma': 'auto', 'kernel': 'linear'}",0.657671,0.612840,0.710169,0.565697,0.677534,0.644782,0.050575,15
4,0.060658,0.007594,0.009486,0.001209,1,scale,rbf,"{'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}",0.853828,0.819888,0.840613,0.835548,0.764314,0.822838,0.031214,1
5,0.092878,0.009385,0.001314,0.001131,1,scale,poly,"{'C': 1, 'gamma': 'scale', 'kernel': 'poly'}",0.846698,0.807698,0.822337,0.818764,0.767978,0.812695,0.025731,8
6,0.088874,0.011342,0.011793,0.001591,1,scale,sigmoid,"{'C': 1, 'gamma': 'scale', 'kernel': 'sigmoid'}",-28.946238,-41.608499,-34.857323,-44.874534,-26.381183,-35.333556,7.093304,19
7,0.094589,0.008024,0.004161,0.003566,1,scale,linear,"{'C': 1, 'gamma': 'scale', 'kernel': 'linear'}",0.657671,0.612840,0.710169,0.565697,0.677534,0.644782,0.050575,15
8,0.141053,0.018819,0.010573,0.001746,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.866490,0.812979,0.832323,0.829786,0.758838,0.820083,0.035211,3
9,0.575861,0.083448,0.003723,0.002331,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.846947,0.805450,0.823072,0.818562,0.768712,0.812549,0.025699,9


In [16]:
filename="SVM_Preprocessed_input_directDeployment.sav"
pickle.dump(grid,open(filename,'wb'))
loaded_model=pickle.load(open("SVM_Preprocessed_input_directDeployment.sav",'rb'))
pre_input=scx.transform([[19,39,0,1,0]])
pre_input

C:\Users\nandh\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[-1.46169465,  1.32337859, -0.89833872,  1.02378711, -0.50466988]])

In [17]:
result=loaded_model.predict(pre_input)
result

array([-0.90825243])

In [21]:
pre_output=scy.inverse_transform([result])
pre_output

array([[2448.54293515]])